In [ ]:
import folium
import ee
import geemap
import geopandas as gpd
import pandas as pd
ee.Authenticate()

# Write your project ID here, in quotes
ee.Initialize(project = "geoemerge-hackathon")

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

start_date = '2025-07-01'
end_date = '2025-07-31'

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.
  Args:
      image (ee.Image): A Sentinel-2 image.
  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [ ]:
data = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/globe_mosquito.zip')
fl = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/florida_boundary.geojson')
data_fl = data.sjoin(fl, how="inner", predicate="within")

region = geemap.geopandas_to_ee(fl)

In [ ]:
lst = (
    ee.ImageCollection('MODIS/061/MOD11A1')
      .filterBounds(region)
      .filterDate(start_date, end_date)
      .select('LST_Day_1km')
      .median()
      .clip(region)
)

rgb = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(region)
      .filterDate(start_date, end_date)
      .map(mask_s2_clouds)
      .median()
      .clip(region)
)
ndvi = rgb.normalizedDifference(['B8', 'B4']).rename('NDVI')

rain = (
    ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
      .filterBounds(region)
      .filterDate(start_date, end_date)
      .select('precipitation')
      .sum()
      .clip(region)
)

In [ ]:
map = folium.Map(location=[28.263363, -83.497652], tiles="Cartodb dark_matter", zoom_start=7)

map.add_ee_layer(lst,
                {'min': 13000.0, 'max': 16500.0,
                'palette': ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                'ff0000', 'de0101', 'c21301', 'a71001', '911003']},
                 "LST")
map.add_ee_layer(ndvi, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI')
map.add_ee_layer(rain, {'min': 0, 'max': 200, 'palette': ['white', 'blue']}, 'Precipitation')

folium.LayerControl(collapsed = False).add_to(map)
display(map)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define a common projection (Sentinel-2)
s2_proj = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .first().select('B1').projection()

# Resample LST and Precipitation to Sentinel-2 resolution
lst_resampled = lst.resample('bilinear').reproject(crs=s2_proj)
rain_resampled = rain.resample('bilinear').reproject(crs=s2_proj)

# Combine layers to one image
combined = ndvi.addBands(lst).addBands(rain)

In [ ]:
# Calculate means
mean_lst = lst.reduceRegion(reducer=ee.Reducer.mean(), geometry=region, scale=1000,
                              maxPixels=1e12).get('LST_Day_1km').getInfo()
mean_rain = rain.reduceRegion(reducer=ee.Reducer.mean(), geometry=region, scale=5566,
                              maxPixels=1e12).get('precipitation').getInfo()

# Define risk classification function
def classify_risk(image):
    ndvi = image.select('NDVI')
    lst = image.select('LST_Day_1km')
    rain = image.select('precipitation')

    low_risk = ndvi.lt(0).And(lst.lt(mean_lst)).And(rain.lt(mean_rain))
    med_risk = ndvi.lte(0.3).Or(lst.eq(mean_lst)).Or(rain.eq(mean_rain))
    high_risk = ndvi.gt(0.3).And(lst.gt(mean_lst)).And(rain.gt(mean_rain))

    # Assign values: 1 = Low, 2 = Medium, 3 = High
    risk = low_risk.multiply(1).add(med_risk.multiply(2)).add(high_risk.multiply(3)).rename('Risk_Level')
    return image.addBands(risk)

# Apply classification
classified = classify_risk(combined)

In [ ]:
# Refresh map
map = folium.Map(location=[28.263363, -83.497652], tiles="Cartodb dark_matter", zoom_start=7)

# Add risk layer
map.add_ee_layer(classified.select('Risk_Level'), {'min': 1, 'max': 3, 'palette': ['green', 'yellow', 'red']}, 'Risk Level')
display(map)

In [ ]:
mosquito = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/globe_mosquito.zip')
land_cover = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/globe_land_cover.zip')

from folium.plugins import HeatMap

# Create empty map zoomed to Florida
map = folium.Map(location=[28.263363, -83.497652], tiles="Cartodb dark_matter", zoom_start=7)

# Add mosquito heatmap
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in mosquito.geometry]
HeatMap(heat_data).add_to(map)

display(map)

In [ ]:
df = pd.read_csv('/hospitals.csv')

In [ ]:
#Combined Environment/Hospital Risk (Environment Weight: 0.7, Hospital Weight: 0.3)
def row_to_feature(row):
    point = ee.Geometry.Point(
        [row['x'], row['y']],
        proj='EPSG:3857'
    )
    return ee.Feature(point)

hospital_features = [row_to_feature(row) for _, row in df.iterrows()]

hospitals = ee.FeatureCollection(hospital_features)
distance_to_hospital = hospitals.distance(
    searchRadius=100000,
    maxError=100
)
distance_miles = distance_to_hospital.multiply(0.000621371)

hospital_risk = (
    distance_miles.where(distance_miles.lt(5), 1)
                  .where(distance_miles.gte(5).And(distance_miles.lt(15)), 2)
                  .where(distance_miles.gte(15), 3)
                  .rename('Hospital_Risk')
)
combined_risk = (
    classified.select('Risk_Level').multiply(0.7)
    .add(hospital_risk.multiply(0.3))
    .round()
    .rename('Combined_Risk')
)

risk_map = folium.Map(
    location=[28.263363, -83.497652],
    tiles="Cartodb dark_matter",
    zoom_start=7
)

risk_map.add_ee_layer(
    combined_risk.select('Combined_Risk'),
    {'min': 1, 'max': 3, 'palette': ['green', 'yellow', 'red']},
    'Combined Risk'
)

display(risk_map)

In [ ]:
import geopandas as gpd
import json
import ee

flbound = gpd.read_file('https://raw.githubusercontent.com/geo-di-lab/emerge-lessons/refs/heads/main/docs/data/florida_boundary.geojson')
fl_json = json.loads(flbound.to_json())
region = ee.FeatureCollection(fl_json).geometry()

risk_levels = [1, 2, 3]

pop_img = ee.ImageCollection("WorldPop/GP/100m/pop") \
    .filter(ee.Filter.eq('year', 2020)) \
    .select('population') \
    .mosaic() \
    .clip(region)

target_prj = ee.Projection('EPSG:4326').atScale(100)

combined_risk_aligned = combined_risk.reproject(crs=target_prj).round()

for level in risk_levels:
    mask = combined_risk_aligned.eq(level)
    masked_pop = pop_img.updateMask(mask)

    stats = masked_pop.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=500,
        maxPixels=1e12
    )

    info = stats.getInfo()
    count = info.get('population')

    if count is not None:
        print(f"Risk Level {level} Total Population: {count:,.0f}")
    else:
        print(f"Risk Level {level} Total Population: 0")


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.x, df.y),
    crs="EPSG:3857"
)

statesurl = "https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip"
state = gpd.read_file(statesurl)

florida = state[state["NAME"] == "Florida"].to_crs("EPSG:4326")
gdf = gdf.to_crs("EPSG:4326")

flhospital = gpd.sjoin(
    gdf, florida, how = "inner", predicate = "within"
)

fl_proj = florida.to_crs("EPSG:3087")
hospitals_proj = flhospital.to_crs("EPSG:3087")

dist20 = 20 * 1609.34
dist30 = 30 * 1609.34

buff20 = hospitals_proj.buffer(dist20).union_all()
buff30 = hospitals_proj.buffer(dist30).union_all()

ring = buff30.difference(buff20)

cov20 = gpd.GeoSeries([buff20], crs="EPSG:3087").clip(fl_proj)
cov30 = gpd.GeoSeries([buff30], crs="EPSG:3087").clip(fl_proj)

fig, ax = plt.subplots(figsize = (10,10))
fl_proj.plot(
    ax=ax,
    color='red',
    edgecolor='black',
    alpha=1
)


cov30.plot(
    ax = ax,
    color = 'yellow',
    label = '20-30 miles to Nearest Hospital'
)

cov20.plot(
    ax = ax,
    color = 'green',
    alpha = 0.4,
    label = '<20 miles to Nearest Hospital'
)

hospitals_proj.plot(ax=ax, color='black', marker = 'D', markersize=20, alpha=0.6, label='Hospitals')

plt.title("20 Mile Hospital Buffer")
plt.show()


# 🦟 GeoEmerge: AI-Powered Vector-Borne Disease Planner
**Hackathon Track:** Intermediate.

**Goal:** To move beyond static risk maps and create an AI that helps tourists and residents make real-time safety decisions.

### 🚀 Key Features:
1.  **AI Trip Planner:** An interactive tool that predicts safety for *future* trips using a "Historical Proxy" algorithm.
2.  **Dynamic Risk Scoring:** Combines Temperature (MODIS), Rainfall (CHIRPS), and Vegetation (Sentinel-2) to calculate a 0-100 Safety Score.
3.  **Florida Deep-Dive:** A granular analysis of mosquito breeding grounds (artificial containers) to guide local resource allocation.

In [ ]:
# 1. Install and Import Libraries
# Installs necessary packages for geolocation and date parsing
!pip install -q geopy dateparser

import ee
import folium
import dateparser
import datetime
from geopy.geocoders import Nominatim
from datetime import timedelta
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Initialize Geocoder
# This service helps convert location names (e.g., "Miami") into coordinates
geolocator = Nominatim(user_agent="safezone_planner_app")

In [ ]:
def get_safety_forecast(geometry, target_date):
    """
    Retrieves satellite data for safety analysis.

    If the target_date is in the future, this function uses data from the
    previous year as a 'Forecast Proxy' to estimate conditions.

    Args:
        geometry (ee.Geometry): The region of interest.
        target_date (datetime): The date to analyze.

    Returns:
        tuple: A tuple containing the processed Earth Engine image (safety score)
               and a descriptive note string.
    """

    # 1. Determine if the request is for the Future
    today = datetime.datetime.now()
    is_future = target_date > today

    # 2. Set the 'Satellite Data Window'
    # We grab a 14-day window around the target date to ensure we get good data coverage
    window_days = 7

    if is_future:
        # FORECAST MODE: Shift back 1 year
        # e.g., User asks for July 2025 -> We read July 2024
        past_date = target_date.replace(year=target_date.year - 1)
        start_str = (past_date - timedelta(days=window_days)).strftime('%Y-%m-%d')
        end_str = (past_date + timedelta(days=window_days)).strftime('%Y-%m-%d')
        display_note = f" <b>Future Forecast:</b> Predicting safety based on historical trends from {start_str}."
    else:
        # HISTORICAL MODE: Look at actual past data
        start_str = (target_date - timedelta(days=window_days)).strftime('%Y-%m-%d')
        end_str = (target_date + timedelta(days=window_days)).strftime('%Y-%m-%d')
        display_note = f" <b>Historical Analysis:</b> Analyzing actual satellite data from {start_str}."

    # 3. Earth Engine Data Retrieval (Dynamic Region)
    # LST (Land Surface Temperature)
    lst = (ee.ImageCollection('MODIS/061/MOD11A1')
           .filterBounds(geometry).filterDate(start_str, end_str)
           .select('LST_Day_1km').mean())

    # Precipitation (Rain)
    rain = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
            .filterBounds(geometry).filterDate(start_str, end_str)
            .select('precipitation').sum())

    # NDVI (Vegetation Index)
    rgb = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterBounds(geometry).filterDate(start_str, end_str)
           .map(lambda img: img.updateMask(img.select('QA60').lt(1))) # Simple cloud mask
           .median())
    ndvi = rgb.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # 4. Calculate Safety Score
    # Normalize values (Heuristics for generic regions)
    lst_score = lst.divide(16000)
    rain_score = rain.divide(200)

    # Risk Formula: Combine factors to estimate mosquito suitability
    mosquito_risk = lst_score.add(rain_score).add(ndvi).divide(3)

    # Invert for Safety Score (0-100)
    safety_score = ee.Image(1).subtract(mosquito_risk).multiply(100).rename('Safety_Score')

    return safety_score.clip(geometry), display_note

In [ ]:
def ask_safezone_trip(user_query):
    """
    Legacy function to process a trip query from a single string.
    Extracts date and location, then performs analysis.
    """
    clear_output()
    print(f" Analyzing trip: '{user_query}'...")

    # Date Extraction
    # We use dateparser to find dates in the text (e.g., "next friday", "august 1st")
    extracted_date = dateparser.parse(user_query, settings={'PREFER_DATES_FROM': 'future'})

    if not extracted_date:
        # Default to "Next Week" if no date found
        extracted_date = datetime.datetime.now() + timedelta(days=7)
        print(" No specific date mentioned. Assuming a trip next week.")
    else:
        print(f" Date Detected: {extracted_date.strftime('%B %d, %Y')}")

    # Location Extraction
    # Geocoding
    try:
        location = geolocator.geocode(user_query) # Try full query first
        if not location:
            clean_query = user_query.replace("next week", "").replace("tomorrow", "").replace("in", "")
            location = geolocator.geocode(clean_query)

        if not location:
            display(HTML(f"<h3 style='color:red'> Could not find location in: '{user_query}'</h3>"))
            return

        print(f" Destination Found: {location.address}")

        # Define Region of Interest
        point = ee.Geometry.Point([location.longitude, location.latitude])
        roi = point.buffer(10000)

        # Run forecast
        safety_img, note = get_safety_forecast(roi, extracted_date)

        # Extract the score
        data = safety_img.reduceRegion(ee.Reducer.mean(), roi, 1000).getInfo()
        score = data.get('Safety_Score', None)

        # Display results
        if score is None:
            display(HTML("<h3 style='color:orange'> Data Unavailable (Too cloudy or out of range)</h3>"))
            return

        # Verdict Logic
        if score > 70:
            verdict = " SAFE TO TRAVEL"
            color = "green"
            desc = "Conditions are unfavorable for mosquitoes."
        elif score > 45:
            verdict = " MODERATE RISK"
            color = "orange"
            desc = "Pack bug spray. Mosquito activity is expected."
        else:
            verdict = " HIGH RISK"
            color = "red"
            desc = "High mosquito activity predicted. Take precautions."

        # The Output Card (No Emojis)
        display(HTML(f"""
        <div style="background-color:#f4f4f4; padding:20px; border-radius:15px; border-left: 10px solid {color}; box-shadow: 2px 2px 10px #ccc;">
            <h2 style="color:{color}; margin-top:0;">{verdict}</h2>
            <h3> Trip to: {location.address.split(',')[0]}</h3>
            <p><b> Travel Date:</b> {extracted_date.strftime('%B %d, %Y')}</p>
            <p><b> Safety Score:</b> {score:.1f} / 100</p>
            <hr>
            <p><i>{desc}</i></p>
            <small>{note}</small>
        </div>
        """))

        # The Map
        m = folium.Map([location.latitude, location.longitude], zoom_start=11, tiles="Cartodb Positron")
        m.add_ee_layer(safety_img, {'min': 0, 'max': 100, 'palette': ['red', 'yellow', 'green']}, 'Safety Forecast')
        folium.Marker([location.latitude, location.longitude], tooltip="Destination").add_to(m)
        display(m)

    except Exception as e:
        print(f"Error: {e}")

## Part 1: The AI Travel Planner
Most satellite data is historical (what happened yesterday), which isn't helpful for a tourist planning a trip for next month.

To solve this, we built a **"Time-Travel" Algorithm**:
* **Input:** User provides a location (e.g., "Miami") and a future date (e.g., "August 1st").
* **Logic:** The system detects the date is in the future and automatically queries the satellite data for the **same week last year**.
* **Output:** A **Safety Score (0-100)** that acts as a data-backed forecast.

> **Tech Stack:** `Earth Engine` (Compute), `Geopy` (Location Parsing), `Dateparser` (NLP), `Folium` (Visualization).

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import dateparser

# 1. Define the Updated Logic Function (Separated Inputs)
def analyze_specific_trip(location_input, date_input):
    """
    Analyzes trip safety based on a specific location and date.
    Updates the UI with a scorecard and map.
    """
    clear_output()
    print(f" Processing trip to '{location_input}' for '{date_input}'...")

    # Validate & Parse Date
    trip_date = dateparser.parse(date_input, settings={'PREFER_DATES_FROM': 'future'})

    if not trip_date:
        display(HTML("<h3 style='color:red'> Error: Could not understand the date. Please try 'next week' or 'YYYY-MM-DD'.</h3>"))
        return

    print(f" Date Confirmed: {trip_date.strftime('%B %d, %Y')}")

    # Geocode Location
    try:
        location = geolocator.geocode(location_input)
        if not location:
            display(HTML(f"<h3 style='color:red'> Error: Could not find location '{location_input}'.</h3>"))
            return

        print(f" Destination Found: {location.address}")

        # Run the Earth Engine Analysis
        # Define Region (10km buffer)
        point = ee.Geometry.Point([location.longitude, location.latitude])
        roi = point.buffer(10000)

        # Get Forecast (using existing function)
        safety_img, note = get_safety_forecast(roi, trip_date)

        # Extract Score
        data = safety_img.reduceRegion(ee.Reducer.mean(), roi, 1000).getInfo()
        score = data.get('Safety_Score', None)

        # Visualize Results
        if score is None:
            display(HTML("<h3 style='color:orange'> No satellite data available for this region.</h3>"))
            return

        # Verdict Logic
        if score > 70:
            verdict = " SAFE TO TRAVEL"
            color = "green"
            desc = "Conditions are unfavorable for mosquitoes."
        elif score > 45:
            verdict = " MODERATE RISK"
            color = "orange"
            desc = "Pack bug spray. Mosquito activity is expected."
        else:
            verdict = " HIGH RISK"
            color = "red"
            desc = "High mosquito activity predicted. Take precautions."

        # Display Result Card (Clean, no emojis)
        display(HTML(f"""
        <div style="background-color:#f4f4f4; padding:20px; border-radius:15px; border-left: 10px solid {color}; margin-top: 10px;">
            <h2 style="color:{color}; margin-top:0;">{verdict}</h2>
            <h3>Trip to: {location.address.split(',')[0]}</h3>
            <p><b>Date:</b> {trip_date.strftime('%B %d, %Y')}</p>
            <p><b>Safety Score:</b> {score:.1f} / 100</p>
            <hr>
            <p><i>{desc}</i></p>
            <small>{note}</small>
        </div>
        """))

        # Display Map
        m = folium.Map([location.latitude, location.longitude], zoom_start=11, tiles="Cartodb Positron")
        m.add_ee_layer(safety_img, {'min': 0, 'max': 100, 'palette': ['red', 'yellow', 'green']}, 'Safety Forecast')
        folium.Marker([location.latitude, location.longitude], tooltip="Destination").add_to(m)
        display(m)

    except Exception as e:
        print(f" Error: {e}")

# 2. Create the New Interface
style = {'description_width': 'initial'}

# Location Widget
loc_widget = widgets.Text(
    placeholder='e.g. Orlando, FL',
    description='<b>Destination:</b>',
    style=style,
    layout=widgets.Layout(width='350px')
)

# Date Widget
date_widget = widgets.Text(
    placeholder='e.g. Next Friday, August 1st',
    description='<b>Travel Date:</b>',
    style=style,
    layout=widgets.Layout(width='350px')
)

# Button
search_btn = widgets.Button(
    description='Analyze Trip',
    button_style='primary',
    icon='plane', # Icon name is standard, not an emoji
    layout=widgets.Layout(width='200px')
)

# Layout Container
ui = widgets.VBox([
    widgets.HTML("<h3>GeoEmerge Trip Planner</h3>"),
    loc_widget,
    date_widget,
    widgets.Box([search_btn], layout=widgets.Layout(margin='10px 0px 0px 0px'))
])

# 3. Connect Button
def on_click(_):
    if loc_widget.value and date_widget.value:
        analyze_specific_trip(loc_widget.value, date_widget.value)
    else:
        print("Please fill in both fields.")

search_btn.on_click(on_click)

# Display
display(ui)

## Part 2: Historical Risk Analysis (Florida)
While the AI Planner helps individuals, city planners need to know where mosquitoes are breeding right now to allocate resources.

In this section, we ingest **Global Mosquito Habitat Data** and filter it down to Florida to answer two questions:
1.  **Root Cause Analysis:** What specific water sources are driving the spread? (e.g., swamps vs. artificial containers).
2.  **Land Cover Correlation:** Are mosquitoes thriving more in residential areas or wild forests?

In [ ]:
# Import libraries for geospatial data analysis
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap

In [ ]:
# Load Mosquito Habitat Mapper data
# This dataset contains observations of mosquito habitats
mosquito_url = 'https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/globe_mosquito.zip'
mosquito = gpd.read_file(mosquito_url)

# Load Land Cover data
# This dataset provides information about the type of land surface
land_cover_url = 'https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/globe_land_cover.zip'
land_cover = gpd.read_file(land_cover_url)

# Load Florida boundary for filtering
# We filter observations to focus only on Florida
fl_url = 'https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/florida_boundary.geojson'
fl = gpd.read_file(fl_url)[['geometry']]

print("Data loaded! Now filtering for Florida...")

# Filter data to only include points within Florida
# We use a spatial join (intersects) to keep only points inside the state boundary
mosquito = gpd.sjoin(mosquito, fl, how="inner", predicate='intersects').drop(columns=['index_right']).reset_index(drop=True)
land_cover = gpd.sjoin(land_cover, fl, how="inner", predicate='intersects').drop(columns=['index_right']).reset_index(drop=True)

print(f"Ready with {len(mosquito)} mosquito records and {len(land_cover)} land cover records.")

In [ ]:
# Analyze water source types
# This helps us identify where mosquitoes are most likely to breed
risk_sources = mosquito['WaterSourceType'].value_counts()
print("Top Mosquito Breeding Sites in Florida:")
print(risk_sources.head(10))

In [ ]:
# Initialize the map centered on Florida
m = folium.Map(location=[28.263363, -83.497652], tiles="Cartodb Positron", zoom_start=7)

# 1. Add a HeatMap layer to show "Hotspots" of mosquito activity
# This visualizes density of observations
heat_data = [[row.geometry.y, row.geometry.x] for idx, row in mosquito.iterrows()]
HeatMap(heat_data, name="Mosquito Hotspots", radius=15).add_to(m)

# 2. Add markers for the most recent observations
# Limit to the first 100 records for performance and clarity
for idx, row in mosquito.head(100).iterrows():
    popup_text = (
        f"<b>Date:</b> {row['MeasuredDate']}<br>"
        f"<b>Source:</b> {row['WaterSourceType']}<br>"
        f"<b>Larvae Count:</b> {row['LarvaeCountProcessed']}"
    )

    # Use a red circle if larvae were found, green if not
    color = 'red' if row['LarvaeCountProcessed'] > 0 else 'green'

    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color=color,
        fill=True,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# Display the map
m

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt

# 2. Load Datasets directly from the source
mosquito_url = 'https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/globe_mosquito.zip'
mosquito = gpd.read_file(mosquito_url)

land_cover_url = 'https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/globe_land_cover.zip'
land_cover = gpd.read_file(land_cover_url)

fl_url = 'https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/florida_boundary.geojson'
fl = gpd.read_file(fl_url)[['geometry']]

# Filter data to only include points within Florida
mosquito = gpd.sjoin(mosquito, fl, how="inner", predicate='intersects').drop(columns=['index_right']).reset_index(drop=True)
land_cover = gpd.sjoin(land_cover, fl, how="inner", predicate='intersects').drop(columns=['index_right']).reset_index(drop=True)

print(f"Data Loaded: {len(mosquito)} mosquito records and {len(land_cover)} land cover records in Florida.")

In [ ]:
# Create a spatial join to link mosquito points with the nearest land cover report
# This helps us understand the environment surrounding a breeding site
integrated_data = gpd.sjoin(mosquito, land_cover[['MucDescription', 'geometry']], how="left", predicate="intersects")

# Compare average larvae counts by land cover type
land_cover_analysis = integrated_data.groupby('MucDescription')['LarvaeCountProcessed'].mean().sort_values(ascending=False)

print("Average Larvae Count by Land Cover Type:")
print(land_cover_analysis)

In [ ]:
# Identify the top water sources for breeding
# Aggregating total larvae count by water source type
water_source_risk = mosquito.groupby('WaterSourceType')['LarvaeCountProcessed'].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
water_source_risk.head(10).plot(kind='bar', color='teal')
plt.title('Top Mosquito Breeding Sites by Water Source')
plt.ylabel('Total Larvae Count')
plt.show()

print(f"The primary breeding source identified is: {water_source_risk.index[0]}")

### The "City Pulse" Map
This final visualization is designed for **Government & Vector Control Units**.

* **Heatmap:** Instantly identifies high-density breeding clusters.
* **"Check Your Yard" Alerts:** We flagged high-risk residential zones where citizens should be notified to empty standing water (e.g., flower pots, tires), which our data identified as the #1 breeding source.

In [ ]:
# Initialize map centered on Florida
sentinel_map = folium.Map(location=[28.2633, -83.4976], tiles="Cartodb dark_matter", zoom_start=7)

# 1. FOR CITIES: Heatmap of Risk
# Shows high-density breeding zones based on larvae counts
heat_data = [[row.geometry.y, row.geometry.x, row['LarvaeCountProcessed']] for idx, row in mosquito.iterrows() if row['LarvaeCountProcessed'] > 0]
HeatMap(heat_data, name="Risk Heatmap (City Use)", radius=15, blur=10).add_to(sentinel_map)

# 2. FOR CITIZENS: Alert System
# We add markers that notify users of high-risk land cover zones
for idx, row in integrated_data.head(50).iterrows():
    risk_level = "HIGH" if row['LarvaeCountProcessed'] > 5 else "MODERATE"
    color = 'red' if risk_level == "HIGH" else 'orange'

    # Popup message
    popup_msg = (
        f"<b>ALERT: {risk_level} RISK ZONE</b><br>"
        f"Land Cover: {row['MucDescription']}<br>"
        f"Common Source: {row['WaterSourceType']}<br>"
        f"<i>Action: Please empty standing water in your yard!</i>"
    )

    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=6,
        color=color,
        fill=True,
        popup=folium.Popup(popup_msg, max_width=250)
    ).add_to(sentinel_map)

folium.LayerControl().add_to(sentinel_map)
sentinel_map